In [1]:
import tensorflow as tf
import numpy as np
import sys

sys.path.append("..")
tf.reset_default_graph()

seed = 1234
dropout = 0.25
rnd = tf.set_random_seed(seed)



/afs/inf.ed.ac.uk/user/s17/s1758150/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 28,28], 'inputs')
targets = tf.placeholder(tf.float32, [None, 47], 'targets')


learning_rate = 0.001
training_steps = 10
batch_size = 50
display_step = 1

num_input = 28
timestep = 28
num_hidden=128
num_classes =47

weights = {'out':tf.Variable(tf.random_normal([num_hidden, num_classes]))
          }
biases = {'out':tf.Variable(tf.random_normal([num_classes]))
         }

x = tf.unstack(inputs, timestep, 1)

lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias = 1.0)

outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

logits = tf.matmul(outputs[-1], weights['out'])+biases['out']
prediction = tf.nn.softmax(logits)

per_datapoint_errors = tf.nn.softmax_cross_entropy_with_logits(\
                        logits = logits, labels =targets)
error = tf.reduce_mean(per_datapoint_errors)


per_datapoint_pred_is_correct = tf.equal(tf.argmax(logits,1),\
                                         tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(per_datapoint_pred_is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(error)



sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

import data_providers as data_providers
from data_provider import ASSISTDataProvider

DATA_DIR = "~/Data/"
train_data = ASSISTDataProvider(DATA_DIR, batch_size=10)

num_epoch = 10

for e in range(num_epoch):
    running_error=0
    for input_batch, target_batch, target_ids_batch in train_data:
        _, batch_error = sess.run([train_step, error], \
                                 feed_dict={inputs: input_batch, targets: target_batch})
        running_error += batch_error
    running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.2f}'.format(e+1, running_error))

ValueError: Cannot feed value of shape (973, 1, 10, 293) for Tensor 'inputs:0', which has shape '(?, 28, 28)'